# Hello world
In this notebook, you'll run a minimal interesting example of `mandala`. It's a
great way to quickly get a feel for the library and play with it yourself!

If you want a more in-depth introduction with a real ML project, check out the
[the next tutorial](01_logistic.ipynb).

## Create the storage and computational primitives
A `Storage` instance is where the results of all computations you run in a
project are stored. Importantly, **the only way to put data into a `Storage` is
to call a function**: you decorate your functions with the `@op` decorator, and
then any time you call them, the inputs and outputs for this call are stored in
the `Storage`.

Go ahead and create a storage and two `@op`-decorated functions:

In [19]:
from mandala.imports import *

# create a storage for results
storage = Storage()

@op # memoization decorator
def inc(x) -> int:
    print('Hi from inc!')
    return x + 1 

@op
def add(x: int, y: int) -> int:
    print('Hi from add!')
    return x + y

### A note on function inputs/outputs
Currently, **`mandala` only supports functions with a fixed number of inputs and
outputs**. To help make this explicit for outputs, you must specify
the number of outputs in the return type annotation. For example, `def f() ->
int` means that `f` returns a single integer, and `def f() -> Tuple[int, int]` means
that `f` returns two integers. Functions that return nothing can leave the
return type annotation empty.

## Your first `mandala`-tracked computation
The main way you use `mandala` is through "workflows", i.e. compositions of
`@op`-decorated functions. Running a workflow for the first time inside a
`storage.run()` block will execute the workflow and store the results in the 
`storage`:

In [20]:
with storage.run():
    x = inc(20)
    y = add(21, x)
    print(y)

Hi from inc!
Hi from add!
ValueRef(42, uid=098eb3a17aa1bb3222230cd64bd9114661f91895947f1b601bc4ff58019775203cc7ce32ce3f3cff1a1e67ea3bdc753032fc0c19311945dcdcb6b9d72f34989a)


Running this workflow a **second** time will not re-execute it, but instead
retrieve the results from the `storage` at each function call along the way:

In [13]:
with storage.run():
    x = inc(20)
    y = add(21, x)
    print(y)

ValueRef(42, uid=098eb3a17aa1bb3222230cd64bd9114661f91895947f1b601bc4ff58019775203cc7ce32ce3f3cff1a1e67ea3bdc753032fc0c19311945dcdcb6b9d72f34989a)


Adding more logic to the workflow will not re-execute the parts that have
already been executed:

In [16]:
with storage.run():
    for a in [10, 20, 30]:
        x = inc(a)
        y = add(21, x)

Finally, code inside the `storage.query()` block can pattern-match to
computational dependencies to retrieve the results of the workflow in a table:

In [18]:
with storage.query() as q:
    a = Q() # a placeholder for a value
    x = inc(a) # same code as above
    y = add(21, x) # same code as above
    df = q.get_table(a.named('a'), x.named('x'), y.named('y'))
df

,a,x,y
0,20,21,42
1,10,11,32
2,30,31,52


Those are the main patterns you need to know to start playing around with
`mandala`! We invite you to go back and modify the code above by creating new
computational primitives and workflows, and see how `mandala` handles it.